In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
valid_data = mnist.validation.next_batch(5000)
test_data = mnist.test.next_batch(10000)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
def flat(x_tensor):
    input_shape = x_tensor.get_shape().as_list()
    flatness = tf.reshape(x_tensor,[-1,input_shape[3]*input_shape[2]*input_shape[1]])
    return flatness

In [3]:
def cnn_model(features,labels):
    input_layer = tf.reshape(features, [-1, 28, 28, 1])
    conv1 = tf.layers.conv2d(inputs=input_layer,filters=64,kernel_size=[3,3],strides=[1,1], padding='SAME',activation=tf.nn.relu)
    conv2 = tf.layers.conv2d(conv1,64,kernel_size=[3,3],strides=[1,1], padding='SAME',activation=tf.nn.relu)
    conv3 = tf.layers.conv2d(conv2,64,kernel_size=[3,3],strides=[1,1], padding='SAME',activation=tf.nn.relu)
    fc1 = tf.layers.dense(flat(conv3),units=512,activation=tf.nn.relu)
    out_layer = tf.layers.dense(fc1, units=10)
    return out_layer

In [10]:
def print_stats(session, feature_batch, label_batch, cost, accuracy):
    """
    Print information about loss and validation accuracy
    : session: Current TensorFlow session
    : feature_batch: Batch of Numpy image data
    : label_batch: Batch of Numpy label data
    : cost: TensorFlow cost function
    : accuracy: TensorFlow accuracy function
    """
    x_valid, y_valid = valid_data
    x_valid = np.reshape(x_valid,[-1,28,28,1])
    accuracy_value = session.run(accuracy, feed_dict={
    x: np.reshape(x_valid,[-1,28,28,1]),
    y: y_valid})
    loss_value = session.run(cost, feed_dict={
    x: feature_batch,
    y: label_batch})
    print("Loss: {:4.4f} Accuracy: {:4.4f}".format(loss_value, accuracy_value))

In [11]:
save_model_path = './'
x = tf.placeholder(tf.float32,shape=(None,28,28,1),name='x')
y = tf.placeholder(tf.float32,shape=(None,10),name='y')
logits = cnn_model(x,y)
loss = tf.losses.softmax_cross_entropy(onehot_labels=y, logits=logits)
cost = tf.reduce_mean(loss)
optimizer = tf.train.AdamOptimizer().minimize(cost)
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')
epochs = 10
valid_data = np.reshape(valid_data,[-1,28,28,1])
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(epochs):
        for batch_i in range(200):
            batch_xs, batch_ys = mnist.train.next_batch(100)
            batch_xs = np.reshape(batch_xs,[-1,28,28,1])
            sess.run(optimizer, feed_dict={x: batch_xs,y: batch_ys})
        print('Epoch {:>2}, CIFAR-10 Batch {}:  '.format(epoch + 1, batch_i), end='')
        print_stats(sess, batch_xs, batch_ys, cost, accuracy)
    saver = tf.train.Saver()
    save_path = saver.save(sess, save_model_path)

Epoch  1, CIFAR-10 Batch 0:  

KeyboardInterrupt: 

In [ ]:
loaded_graph = tf.Graph()
test_data = np.reshape(test_data,[-1,28,28,1])
test_feature, test_label = test_data
with tf.Session(graph=loaded_graph) as sess:
    loader = tf.train.import_meta_graph(save_model_path + '.meta')
    loader.restore(sess, save_model_path)
    loaded_x = loaded_graph.get_tensor_by_name('x:0')
    loaded_y = loaded_graph.get_tensor_by_name('y:0')
    loaded_acc = loaded_graph.get_tensor_by_name('accuracy:0')
    test_batch_acc_total = 0
    test_batch_count = 0
    for test_feature_batch, test_label_batch in 
        test_batch_acc_total += sess.run(
            loaded_acc,
            feed_dict={loaded_x: test_feature, loaded_y: test_label})
        test_batch_count += 1
    print('Testing Accuracy: {}\n'.format(test_batch_acc_total/test_batch_count))